In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn import svm
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from collections import Counter
import nltk

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=0e63ac66f618328e684cbfe197131a915a0cf5372ccc4df518c672a16af5137e
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


# HELPER FUNCTIONS

In [ ]:
nltk.download('stopwords')
stopwords_list = stopwords.words('english')
stopwords_list.extend(['nalla','ena','unaku','per','irukanga','panna','yarum','mattum','ivan','ada','pesa','unakku','k','sari','idhu','vida','vittu','enga','yen','ithu','poda','dey','irundhu','ya','la', 'u','r','s','bro','da','dei','dai','nu','ah','nee','ni','illa','un','ok','na','pls','ur','unga']) 
stopwords_list.extend(['🥰','indha','antha','vera','iruka','pola','innum','avan','summa','ellam','thaan','romba','❤️','ana','ama','apdi','ithula','po','evlo','eruku','irukum','nama','enna','va','hi','h','ku','iruku','naa','va','oru','athu','avanga','neenga','tha','en','di','dhan','ne','ella','intha']) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
label = { 'None-of-the-above': 0, 'Transphobic':1, 'Counter-speech':2, 'Misandry':3,
       'Homophobia':4, 'Hope-Speech':5, 'Xenophobia':6, 'Misogyny':7}

In [ ]:
def tolower(text):
  return text.lower()

def removepunctuation(text):
    clean="".join([i for i in text if i not in string.punctuation])
    return clean

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def removestopwords(text):
  clean = " ".join([word for word in text.split() if word not in stopwords_list])
  return clean

In [ ]:
def preprocess(df):
  df['text']= df['text'].apply(lambda x:tolower(x))
  df['text']= df['text'].apply(lambda x:removepunctuation(x))
  df['text']= df['text'].apply(lambda x:deEmojify(x))
  df['text']= df['text'].apply(lambda x:removestopwords(x))
  df['category'] = df['category'].map(label)
  return df

In [ ]:
def tokenize(text):
  return text.split()

def tokenizedf(df):
  df['token']= df['text'].apply(lambda x:tokenize(x))
  df['len'] = df.text.apply(lambda x : len(x))
  return df

In [ ]:
def stats(value):
    if value == 0:
        return 0
    else:
        return 1

In [ ]:
def vocab(df):
  return df.groupby('category')['text'].apply(lambda x: Counter(" ".join(x).split()).most_common(100))

In [ ]:
def getVocab(df):
  df_vocab = vocab(df)
  processed = list(set([key for groups in df_vocab for key, value in groups]))
  return processed

In [ ]:
def fitTfidfVectrain(df, tfidf_vec):
  x_df = tfidf_vec.fit_transform(df['text'])
  return x_df

In [ ]:
def getXtrain(model, df, tfidf_vec):
  model_tf_x = fitTfidfVectrain(df, tfidf_vec)
  model_val_x = model.encode(df['text'].tolist())
  Xval = [np.append(list1, list2) for list1, list2 in zip(model_val_x, model_tf_x.toarray())]
  return Xval

In [ ]:
def fitTfidfVectest(df, tfidf_vec):
  x_df = tfidf_vec.transform(df['text'])
  return x_df

In [ ]:
def getXtest(model, df, tfidf_vec):
  model_tf_x = fitTfidfVectest(df, tfidf_vec)
  model_val_x = model.encode(df['text'].tolist())
  Xval = [np.append(list1, list2) for list1, list2 in zip(model_val_x, model_tf_x.toarray())]
  return Xval

In [ ]:
def getY(df):
  return df['category']

# MODELS

In [ ]:
labse_embedding_model = SentenceTransformer('sentence-transformers/LaBSE')

# DATASETS AND PREPROCESSING

In [ ]:
df_train = pd.read_csv('/content/drive/Shareddrives/NLP Research work - PANDAS team/ACL/Datasets/ACD/ta-en-misogyny-train.csv',sep='\t', names = ["category", "text"])
df_train.text=df_train.text.astype(str)
train = df_train
train = preprocess(df_train)

In [ ]:
df_test = pd.read_csv('/content/drive/Shareddrives/NLP Research work - PANDAS team/ACL/Datasets/ACD/ta-en-misogyny-dev.csv',sep='\t', names = ["category", "text"])
df_test.text=df_test.text.astype(str)
test = df_test
test = preprocess(test)

In [ ]:
train

,category,text
0,0,enaku unmaikum aluha wantu thirunangaigal thei...
1,1,superstar vijay arivuketta polu thappu pavam a...
2,0,ugka smile cute
3,0,anna waiting 🥰🥰🥰
4,0,yanda tamilnadu evvalavo pirachana athalam vit...
...,...,...
5943,0,noq day caste religious certificate
5944,0,mimicry escape aitaru
5945,0,rajesh age
5946,2,videos nallarukku dont prank eena varavanga ep...


In [ ]:
label

{'None-of-the-above': 0,
 'Transphobic': 1,
 'Counter-speech': 2,
 'Misandry': 3,
 'Homophobia': 4,
 'Hope-Speech': 5,
 'Xenophobia': 6,
 'Misogyny': 7}

In [ ]:
df_train[df_train.category == 7]

,category,text
88,7,cassia 42 sister paavm amma appa manasa purinj...
97,7,oaththa kundu thakkale paththaka 🤢🤢
110,7,loose kooda interveiw panreenga kuruttu koothi...
130,7,enaku doubt iva aambala ila pombala
133,7,ivalukaluku lam kalyanam nadakakudathu
...,...,...
5845,7,yadi thevadiya niraya sexual disease varum med...
5909,7,priya priya thayvidiyaa mundai oththaa aripped...
5912,7,kandipa pondati pecha kelunga kooda porandha t...
5914,7,meme trolls trolls endi poonde mariyade koodut...


In [ ]:
train.category.value_counts() 

0    3720
3     830
2     348
6     297
5     213
7     211
4     172
1     157
Name: category, dtype: int64

In [ ]:
weight = [3100, 157, 348, 830, 172, 213, 297, 211]
normweights = [float(i)/sum(weight) for i in weight]

In [ ]:
df1 = train.loc[train.category == 0 ].copy()

In [ ]:
df1

,category,text
0,0,enaku unmaikum aluha wantu thirunangaigal thei...
2,0,ugka smile cute
3,0,anna waiting 🥰🥰🥰
4,0,yanda tamilnadu evvalavo pirachana athalam vit...
6,0,l v konja varshathula yaaru agadhi aah sutha p...
...,...,...
5940,0,meimozhi pandian manasu kashta padum ninachu p...
5943,0,noq day caste religious certificate
5944,0,mimicry escape aitaru
5945,0,rajesh age


In [ ]:
df0 = df1.head(3100)

In [ ]:
df2 = train.loc[train.category != 0 ].copy()

In [ ]:
df2

,category,text
1,1,superstar vijay arivuketta polu thappu pavam a...
5,1,night fulla mater alaiuradhu day fulla pichaik...
9,2,pradeep kodi maataanga nenaikireengala ellaaru...
10,3,yena public pudikalayadei arivu ellathavane at...
12,4,515 paiyyan straight girl friend gay illai ava...
...,...,...
5934,3,hiphopgamer92 thevdiaaa paiyaaa pottaa amaaa j...
5938,3,mendal epo ponnunga ambala rights thanthangale...
5941,3,endha thevidiya payalavairamuthuva thaane solr...
5942,7,nadu roomba irunthathu epo ponnunga ambala rig...


In [ ]:
train = pd.concat([df0, df2], axis=0)
train = train.sample(frac=1)

In [ ]:
train

,category,text
2007,3,aandal thayai ngra apo appan kuda paduthala ec...
2479,0,channel mobile number sent pana
4759,0,laa bromunnadi erunthae irukuippo neraya aiduchi
2017,4,anna homosexualitya eppadi cure pandradhu vide...
4141,0,storm breaker unaiya onume solala gounder vara...
...,...,...
622,3,cena devi ombodhunaan thirunelveli kaaran diun...
478,0,richard fredrick unaiku thruma
23,3,venna admin karu palaniyappan sunni
9,2,pradeep kodi maataanga nenaikireengala ellaaru...


In [ ]:
test

,category,text
0,0,black saree hot
1,2,halo first ayunga kai thatti amount keatkyrang...
2,0,hello manithana sollunga pramanana solla vanam
3,6,china kaaranai kalaipan daily enjoying 2016
4,0,onnota ponnu ipdi vituviya
...,...,...
1483,1,9 usu vinoth
1484,0,love panuradhaaaa soliyaeeee cover panura niiii
1485,0,part 2 aippa varum
1486,3,jii nariya thetunga andha potta gopiya


#VOCABULARY AND FEATURE EXTRACTION

In [ ]:
vocab = getVocab(train)

In [ ]:
vocab[3]

'badu'

#TRAINING - Level 1 - max classification

In [ ]:
tfidf_vec = TfidfVectorizer(analyzer = 'word', vocabulary = vocab)

In [ ]:
Xtrain = getXtrain(labse_embedding_model, train, tfidf_vec)
Xtest = getXtest(labse_embedding_model, test, tfidf_vec)

In [ ]:
y_train = getY(train)
y_test = getY(test)

In [ ]:
svm_model = svm.SVC(kernel='linear', C=1, verbose=True, probability = True)
svm_model.fit(Xtrain, y_train)

[LibSVM]

SVC(C=1, kernel='linear', probability=True, verbose=True)

Original

In [ ]:
ypred = svm_model.predict(Xtest)
print(classification_report(y_test, ypred))

              precision    recall  f1-score   support

           0       0.77      0.94      0.85       919
           1       0.47      0.23      0.31        40
           2       0.42      0.33      0.37        95
           3       0.71      0.57      0.63       218
           4       0.75      0.35      0.48        43
           5       0.38      0.11      0.17        53
           6       0.82      0.59      0.68        70
           7       0.80      0.24      0.37        50

    accuracy                           0.74      1488
   macro avg       0.64      0.42      0.48      1488
weighted avg       0.72      0.74      0.71      1488



Linear model with Intercept

In [ ]:
svm_model = svm.LinearSVC(C=1, verbose=True, fit_intercept = True, intercept_scaling = 1)
svm_model.fit(Xtrain, y_train)
ypred = svm_model.predict(Xtest)
print(classification_report(y_test, ypred))

[LibLinear]              precision    recall  f1-score   support

           0       0.80      0.90      0.85       919
           1       0.30      0.20      0.24        40
           2       0.45      0.31      0.36        95
           3       0.61      0.58      0.60       218
           4       0.56      0.33      0.41        43
           5       0.30      0.17      0.22        53
           6       0.73      0.69      0.71        70
           7       0.50      0.30      0.37        50

    accuracy                           0.73      1488
   macro avg       0.53      0.43      0.47      1488
weighted avg       0.70      0.73      0.71      1488



Linear model with modified Class Weights

In [ ]:
weight

[3100, 157, 348, 830, 172, 213, 297, 211]

In [ ]:
svm_model = svm.LinearSVC(C = 1, class_weight = {0: 0.09, 1: 0.13, 2: 0.13, 3: 0.13, 4: 0.13, 5: 0.13, 6: 0.13, 7: 0.13}, verbose = True, fit_intercept = True, intercept_scaling = 1, probability = True)
svm_model.fit(Xtrain, y_train)
ypred = svm_model.predict(Xtest)
print(classification_report(y_test, ypred))

[LibLinear]              precision    recall  f1-score   support

           0       0.83      0.86      0.84       919
           1       0.33      0.30      0.32        40
           2       0.41      0.38      0.40        95
           3       0.58      0.63      0.60       218
           4       0.59      0.40      0.47        43
           5       0.30      0.26      0.28        53
           6       0.65      0.67      0.66        70
           7       0.54      0.28      0.37        50

    accuracy                           0.72      1488
   macro avg       0.53      0.47      0.49      1488
weighted avg       0.71      0.72      0.71      1488



In [ ]:
analysis = test
test["predicted"] = ypred

In [ ]:
test[test.category != test.predicted]

,category,text,predicted
1,2,halo first ayunga kai thatti amount keatkyrang...,3
11,7,guru murthi dhevudiyalukku porantha dhevudiya ...,3
14,3,arumaiyana speech akka karu nai pudunguvan ipo,0
15,6,ivanugaluki ithe velaiya pochu anga kandu pudi...,1
31,7,thalaivi மீரா மிதுன் oda handbag secret reveal...,0
...,...,...,...
1459,0,ipdi channel vaal naal paathathu ila harts guy...,5
1461,3,thaasi magananalu tamil thirukuralai pizhaiall...,0
1464,5,naal life ungalukku pudiccha maari maarum,0
1471,3,avaru edho 1 time decent soliruparu kevalama t...,0


# Lime XAI

In [ ]:
!pip install lime

In [ ]:
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline
from lime.lime_text import IndexedString,IndexedCharacters
from lime.lime_base import LimeBase
from sklearn.linear_model import Ridge, lars_path
from lime.lime_text import explanation
from functools import partial
import scipy as sp
from sklearn.utils import check_random_state

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
class InputTransformer(BaseEstimator, TransformerMixin):
  def __init__(self,labse = labse_embedding_model, df = train, vec = tfidf_vec):
    self.labse = labse
    self.df = df
    self.vec = vec
  def fit(self, x, y=None):
    return self
  def transform(self, x):
    model_tf_x = self.vec.transform(x)
    model_val_x = self.labse.encode(x)
    Xval = [np.append(list1, list2) for list1, list2 in zip(model_val_x, model_tf_x.toarray())]
    return Xval

In [ ]:
iptransformer = InputTransformer(labse = labse_embedding_model, df = train, vec = tfidf_vec)

In [ ]:
x = train["text"][1]

In [ ]:
iptransformer.transform([x])

In [ ]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(iptransformer, svm_model)

In [ ]:
class_names = svm_model.classes_
explainer = LimeTextExplainer(class_names = class_names)

#Checking indexes

In [ ]:
train.loc[(train['category'] == 4)]

#test

In [ ]:
idx = train.index[257]

In [ ]:
exp = explainer.explain_instance(train["text"][idx], c.predict_proba, num_features = 6, top_labels=23)

In [ ]:
print("Question: \n", train["text"][idx])
print("Probability (None-of-the-above) =", c.predict_proba([train["text"][idx]])[0, 0])
print("Probability (Transphobia) =", c.predict_proba([train["text"][idx]])[0, 1])
print("Probability (Counter-Speech) =", c.predict_proba([train["text"][idx]])[0, 2])
print("Probability (Misandry) =", c.predict_proba([train["text"][idx]])[0, 3])
print("Probability (Homophobia) =", c.predict_proba([train["text"][idx]])[0, 4])
print("Probability (Hope-Speech) =", c.predict_proba([train["text"][idx]])[0, 5])
print("Probability (Xenophobia) =", c.predict_proba([train["text"][idx]])[0, 6])
print("Probability (Misogyny) =", c.predict_proba([train["text"][idx]])[0, 7])
print("True Class is:", train["category"][idx])

In [ ]:
print(exp.as_list(label = 4))

In [ ]:
print(exp.as_list)
exp.show_in_notebook(text=train["text"][idx])

Weights

In [ ]:
tfidf_vec

In [ ]:
vect = fitTfidfVectrain(train, tfidf_vec)

In [ ]:
vect

In [ ]:
vect.toarray()